In [1]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Copy relevant data over (here, we'll be using the raw dataset).
!cp /content/drive/MyDrive/Kalman_Filter/combined_sentinel_raw.csv /content/ # Note! This is the raw dataset, not the 2 std dev dataset.

In [8]:
# Import relevant libraries. 
import pandas as pd
import numpy as np

In [4]:
# Step 0 is always loading and isolating the relevant data.
# Note! You can also use the cleaned version of the satellite readings. 
df = pd.read_csv("combined_sentinel_raw.csv")

In [5]:
df = df.drop(columns=['Unnamed: 0'], axis=1)
df.head(2)

,date,lake,latitude,longitude,orbit,water_level
0,20160403,Athabasca,59.244637,-108.047469,303,208.303892
1,20160403,Athabasca,59.247528,-108.049405,303,208.131251


In [6]:
# Isolate the readings from Lake Winnipeg
# Note! You can isolate any of the lakes. 
win = df[df['lake'] == 'Winnipeg']
win.head(2)

,date,lake,latitude,longitude,orbit,water_level
422462,20160401,Winnipeg,51.860057,-96.819518,283,217.4011
422463,20160401,Winnipeg,51.857120,-96.820913,283,217.5197


##### As the water level readings from a satellite on a given pass (which takes roughly a 1-3 seconds), any variation would be error in the measurements. 

In [12]:
win_error = [] # Create a list where each track's error can be collected. 
  
for i in win.date.unique(): # Collect the error of each track. 
  min = win[win['date'] == i].water_level.min()
  max = win[win['date'] == i].water_level.max() 
  error = max - min
  win_error.append(error)

avg_error = np.mean(win_error)
max_error = np.max(win_error) 
min_error = np.min(win_error) 

print("The average error (in meters) is:", avg_error)
print("The maximum error is", max_error) # 
print("The minimum error is", min_error)

The average error (in meters) is: 8.41012065934066
The maximum error is 65.76687520000002
The minimum error is 0.3317239999999799


In [10]:
# Create a dataframe from the daily errors. 
winnipeg = pd.DataFrame({
    "error": win_error
})

winnipeg.head()

,error
0,0.348200
1,2.112597
2,0.561211
3,0.495355
4,0.682247


In [11]:
# Turn the dataframe into a csv file, for processing in the neural network. 
winnipeg.to_csv("winnipeg.csv")

In [13]:
!cp '/content/winnipeg.csv' '/content/drive/MyDrive/Kalman_Filter/'